In [ ]:
import iris_pipeline
iris_pipeline.monkeypatch_jwst_datamodels()

import numpy as np

In [ ]:
iris_pipeline.__file__

The purpose of this notebook is to create a dataset with a full frame and relative subarrays

## Prepare the raw science frame

* Load raw science frame
* Add `SUBARR_MAP` extension

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from test_utils import get_data_from_url
raw_science_filename = get_data_from_url("17903858")
im = iris_pipeline.datamodels.IRISImageModel(raw_science_filename)

In [ ]:
s1 = 300
s2 = 100
subarray_maps_metadata = []
# square subarray
subarray_maps_metadata.append({"xstart":2048-s1//2, "ystart":2048-s1//2, "xsize":s1, "ysize":s1})  
# test a rectangular subarray
subarray_maps_metadata.append({"xstart":1040-s2//2, "ystart":1040-s2//2, "xsize":s2, "ysize":s2//2})  

In [ ]:
plt.imshow(im.data);

In [ ]:
def set_subarray_mask(mask_array, subarray_index, xstart, ystart, xsize, ysize):
    xstart = xstart - 1
    ystart = ystart - 1
    mask_array[ystart:ystart+ysize, xstart:xstart+xsize] =  subarray_index                 

## Create subarrays

* slice `data`, `dq`, `err`
* set metadata

In [ ]:
def slice_subarray_mask(mask_array, xstart, ystart, xsize, ysize):
    xstart = xstart - 1
    ystart = ystart - 1
    return mask_array[ystart:ystart+ysize, xstart:xstart+xsize].copy()              

In [ ]:
subarray_im = {}
for i, shape in enumerate(subarray_maps_metadata):
    subarray_im[i+1] = im.copy()
    for ext in ["data", "dq", "err"]:
        subarray_im[i+1][ext] = slice_subarray_mask(im[ext], **shape)

In [ ]:
for i, shape in enumerate(subarray_maps_metadata):
    subarray_im[i+1].meta.subarray.name = "CUSTOM"
    subarray_im[i+1].meta.subarray.id = i+1  
    for k, v in shape.items():
        setattr(subarray_im[i+1].meta.subarray, k, v)

In [ ]:
subarray_im[1].meta.subarray.xsize

In [ ]:
im.subarr_map = np.zeros_like(im.data)

for i, shape in enumerate(subarray_maps_metadata):
    set_subarray_mask(im.subarr_map, subarray_index=i+1, **shape)

In [ ]:
plt.imshow(subarray_im[1].data);

In [ ]:
plt.imshow(subarray_im[2].data);

In [ ]:
plt.imshow(im.subarr_map);

In [ ]:
im.data[im.subarr_map != 0] = np.nan

In [ ]:
full_frame_filename = "data/raw_science_frame_sci_with_subarrays.fits"
im.write(full_frame_filename)

In [ ]:
subarray_filename_template = "data/raw_science_frame_sci_subarray_{}.fits"
for k,v in subarray_im.items():
    v.write(subarray_filename_template.format(k))

In [ ]:
plt.imshow(subarray_im[1].data)

In [ ]:
!ls -lah data/*science*sub*fits

## Run the image2 pipeline

In [ ]:
pipeline = iris_pipeline.Image2Pipeline()

reduced_full_frame = pipeline.call(full_frame_filename,
             config_file="data/image2_iris.cfg")[0]

In [ ]:
reduced_full_frame.meta.subarray.id = 0

In [ ]:
reduced_full_frame.write(full_frame_filename.replace("raw", "reduced"))

In [ ]:
full_frame_filename.replace("raw", "reduced")

In [ ]:
reduced_subarray = {}
for i in [1,2]:
    reduced_subarray[i] = pipeline.call(subarray_filename_template.format(i),
             config_file="data/image2_iris.cfg")[0]
    reduced_subarray[i].write(subarray_filename_template.format(i).replace("raw", "reduced"))


## Check the output has correct metadata

In [ ]:
i = 1
for full_frame_meta, each_input in zip(reduced_full_frame.meta.subarray_map, subarray_maps_metadata):
    assert full_frame_meta.instance == each_input
    assert reduced_subarray[i].meta.subarray.instance == dict(name="CUSTOM", id=i, **each_input)
    i += 1

In [ ]:
plt.imshow(reduced_full_frame.data);

In [ ]:
plt.imshow(reduced_subarray[1].dq)

In [ ]:
plt.imshow(reduced_subarray[1].data);

In [ ]:
plt.imshow(reduced_subarray[2].data);